In [1]:
import csv
import re

def sql_to_csv(sql_file, csv_file, table_name=None):
    insert_pattern = re.compile(
        r"INSERT INTO\s+`?(?P<table>\w+)`?\s+VALUES\s*(?P<values>.+);", re.IGNORECASE | re.DOTALL
    )
    value_pattern = re.compile(r"\((.*?)\)")
    
    with open(sql_file, 'r', encoding='utf-8') as f:
        content = f.read()
    
    # Filter insert statements
    inserts = insert_pattern.findall(content)
    
    rows = []
    for table, values_str in inserts:
        if table_name and table != table_name:
            continue
        matches = value_pattern.findall(values_str)
        for match in matches:
            row = [x.strip().strip("'").replace("\\'", "'") for x in match.split(',')]
            rows.append(row)

    # Guess column count from first row
    max_columns = max(len(row) for row in rows)

    # Write CSV
    with open(csv_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        for row in rows:
            row += [''] * (max_columns - len(row))  # pad short rows
            writer.writerow(row)

    print(f"Conversion complete. Saved to {csv_file}")

# Usage example
sql_to_csv('db_dump.sql', 'db_dump.csv')


Conversion complete. Saved to db_dump.csv
